In [33]:
import os 
from pathlib import Path

import pandas as pd
import numpy as np


In [34]:
from sbscorer.sblegos.TransactionAnalyser import TransactionAnalyser as txa

In [35]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
PATH_TO_EXPORT = os.path.join(current_dir.parent.parent, 'tx_data', FOLDER_NAME)
PATH_TO_GR18 = os.path.join(current_dir.parent.parent, FOLDER_NAME)
voter_tx_file_name = 'gg18_votes.csv'
passport_file_name = 'gg18_voter_passports.csv'

tx_file_name = 'gr_18_optimism_all_tx.csv'

In [9]:
votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))
passports = pd.read_csv(os.path.join(PATH_TO_GR18, passport_file_name))

C:\Users\DELL Admin\AppData\Local\Temp\ipykernel_15428\1560736169.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))


In [37]:
votes.round_name.unique()

array(['Climate Round', 'Web3 Open Source Software',
       'Web3 Community and Education ',
       'Global Chinese Community for Public Goods - GR18', 'Web3 Social',
       'Zuzalu Continuous Innovation', 'Meta Pool LatAm GG18',
       'ReFi DAO Local Node ', 'Token Engineering',
       'Ethereum Infrastructure', 'Arbitrum Domain Round'], dtype=object)

In [38]:
votes.shape

(328389, 15)

In [41]:
votes = votes[np.logical_not(np.logical_or(votes['round_name'] == 'Ethereum Infrastructure', votes['round_name'] == 'Arbitrum Domain Round'))]

In [42]:
votes.shape

(258994, 15)

In [43]:
round_names = votes['round_name'].unique()
round_names

array(['Climate Round', 'Web3 Open Source Software',
       'Web3 Community and Education ',
       'Global Chinese Community for Public Goods - GR18', 'Web3 Social',
       'Zuzalu Continuous Innovation', 'Meta Pool LatAm GG18',
       'ReFi DAO Local Node ', 'Token Engineering'], dtype=object)

In [44]:
# Load tx data
df_tx = pd.read_csv(os.path.join(PATH_TO_EXPORT, tx_file_name))
df_tx.head()

,tx_hash,block_timestamp,from_address,to_address,gas_limit,gas_used,tx_fee,eth_value,__row_index
0,0x863c5b543d29b329891fdec341845a73c2525e6dc212...,2023-08-24T20:22:03.000Z,0xf4af0314a06ea7376da5bdecc33f6f9800f1ae28,0x7f5c764cbc14f9669b88837ca1490cca17c31607,53067,53067,0.000029,0.000000,0
1,0xfdffb6eb77e72ca9f54297ebcc7644ea55db521f64c7...,2023-08-24T22:05:41.000Z,0x87dbffd0ed90b2252856faf753437a7bc9fcfa7b,0xb009b5707e6e0dc9830a1262de37f055972e6480,219895,139765,0.000058,0.000777,1
2,0x7b416f625fd29c2da17a7ad35b1c6dc7e0cf5b548249...,2023-08-24T16:10:17.000Z,0x532c1fc2940ec38d5c5e7fc8b8aa4aaddcaa17d3,0x46ce3b08bdef4461fb444d3746fd97f0af11dccf,731592,467423,0.000093,0.000000,2
3,0xbed4ce09f5f5c4d7aac7e310dfce11d94bf86f7da6d5...,2023-08-24T16:09:33.000Z,0x0596c217a79ea00e5dafc43778138ea61fd67ee2,0x2e42f214467f647fe687fd9a2bf3baddfa737465,119625,119625,0.000087,0.000000,3
4,0xb60502458c8de31606b84e0aed102ac83629726d8734...,2023-08-24T22:08:15.000Z,0xc1991c8bda29507991eeb230ff2063c2ea74a34c,0xa062ae8a9c5e11aaa026fc2670b0d65ccc8b2858,246899,229592,0.000070,0.000000,4


In [45]:
df_tx.drop(columns=['__row_index'], inplace=True)

In [46]:
# Choose round name to compute features
round_name = round_names[0]

In [47]:
CONTRACT_CREATION_AD_NAME = '0x_contract_creation'

In [48]:
df_tx.loc[df_tx['to_address'].isna(), 'to_address'] = CONTRACT_CREATION_AD_NAME

In [49]:
df_tx.isna().sum()

tx_hash            0
block_timestamp    0
from_address       0
to_address         0
gas_limit          0
gas_used           0
tx_fee             0
eth_value          0
dtype: int64

In [50]:
df_tx.shape

(6894506, 8)

In [51]:
unique_voters = votes['voter'].unique()
print(f'Number of voters in round: {len(unique_voters)}')

Number of voters in round: 44957


In [52]:
df_tx_EOA_FROM = df_tx.copy()
df_tx_EOA_FROM['EOA'] = df_tx_EOA_FROM['from_address']

In [53]:
df_tx_EOA_TO = df_tx.copy()
df_tx_EOA_TO['EOA'] = df_tx_EOA_TO['to_address']

In [54]:
df_tx_eoa = pd.concat([df_tx_EOA_FROM, df_tx_EOA_TO])

In [55]:
df_tx_eoa.shape

(13789012, 9)

In [56]:
df_tx_eoa = df_tx_eoa.loc[df_tx_eoa['EOA'] != CONTRACT_CREATION_AD_NAME, :]

In [57]:
df_tx_eoa.shape

(13787932, 9)

In [58]:
lowercase_converter = np.vectorize(str.lower)
unique_voters = lowercase_converter(unique_voters)
print(f'Number of voters in round: {len(unique_voters)}')

Number of voters in round: 44957


In [59]:
unique_eoa = df_tx_eoa['EOA'].unique()

In [60]:
print(f'Number of unique EOA: {len(unique_eoa)}')

Number of unique EOA: 140257


In [61]:
diff = np.setdiff1d(unique_voters, unique_eoa) # that are in a but not in b
print(len(diff))
diff

17


array(['0x0000024bc043e565527f26318f7bb5dbae457a36',
       '0x105f63328d809455f149350348a5a615a92e7380',
       '0x11e223c49b3bc43c011ef62f79ce81bfd29d3b74',
       '0x264d4c968832671d750512051fcc4af6ca002db9',
       '0x528fd0edf2d42b4691d6f2a3a040f1610510065c',
       '0x5f80335076b788ba0a898b4d8d88fcc967720bff',
       '0x8efa270c263990f64eb3b1261f9a7925c1fdb878',
       '0x968a0e5603c5d4dbf24cbd7df562921d158ad19c',
       '0xa1ff7552cc06cbd4bff90e478857fd2a2553b66e',
       '0xb0cbfba1ebb77c6639e2b9513c286a77392c7b8b',
       '0xb29e0c1906ebb53f8d9c81625bcddc085e057606',
       '0xc31442c950e7151ded79386a506be868f61792ee',
       '0xcaebd06d75b5f8c77a73df27ab56964ccc64f793',
       '0xcc596bc6753cb82e4383b83bda25b1133a84a5b3',
       '0xd3055381ce349b4cb7116a0b3fab762c3f16fa45',
       '0xdd903d23af089a4d2d257ef2a4d34e755813da7b',
       '0xed3d344d9af0d139018c36255da5371b2ff458c7'], dtype='<U42')

In [62]:
#faster?
count = np.sum(~np.in1d(unique_voters, unique_eoa))
print(count)

17


In [63]:
diff_eoa = np.setdiff1d(unique_eoa, unique_voters)
print(len(diff_eoa))

95317
